#### 1. Introducción

El presente trabajo busca analizar desde la perspectiva de las redes neuronales convolucionales el dataset conformado por 1125 imágenes relativos al clima. El mismo fue descargado del repositorio 'Mendeley Data' (url: https://data.mendeley.com/datasets/4drtyfjtfy/1). El dataset  contiene imágenes de cielos nublados, lluviosos, soleados, y de amaneceres.
El objetivo es aplicar las redes neuronales, de manera de predecir el tipo de cielo de cada imagen.

El primer paso ha sido crear carpetas de 'train' y 'test' para cada tipo de cielo, con una proporción 0.8 y 0.2 respectivamente. Luego se han cargado dichas carpetas a google drive.

In [1]:
# Monto la unidad del drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 2. Convolución 

Carga de librerías e inicialización de la CNN

In [2]:
# Carga de librerías necesarias
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


In [3]:
# Inicialización de la CNN (no requiere la introducción de parámetros)
classifier = Sequential()

A partir de la imagen de entrada y utlizando diversos detectores de rasgos, se procede a la creación de los mapas de características que conforman la capa de convolución.
Tal como se puede ver debajo, se han especificado 32 detectores de rasgos, de tamaño 3x3.

In [4]:
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), 
                      input_shape = (64, 64, 3), activation = "relu"))

A continuación se aplica un max pooling, de manera de reducir la complejidad y permitir a la red neuronal no solo aprender posiciones y colores, sino también transformaciones y escalados de imágenes.

In [5]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

Se añade una segunda capa de convolución y max pooling de iguales características a la ya añadida

In [6]:
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))

Se añade una tercera capa con 64 detectores de rasgos

In [7]:
classifier.add(Conv2D(filters = 64,kernel_size = (3, 3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2)))

Se sigue con el aplanado de los datos (flattening), pasando de las capas pooling a un vector que se lleve toda la información en una sola dimensión vertical que servirá de capa de entrada de la RNA.

In [8]:
classifier.add(Flatten())

Se concluye esta sección realizando la full conection, detallando una función de activación de la capa oculta de tipo 'rectificador lineal unitario'. De igual modo, se utiliza para la capa de salida una función 'softmax'. 

In [9]:
classifier.add(Dense(units = 64, activation = "relu"))
classifier.add(Dense(units = 4, activation = "softmax"))

Compilación de la CNN

In [10]:
classifier.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

#### 3. Ajustar la CNN a las imágenes a entrenar

Se procede a cargar las imágenes mediante la técnica de aumento de imagen, de modo de evitar el sobreajuste. 
Mediante esta técnica se realizan transformaciones aleatorias a las imagenes, lo que permite 'multiplicar' artificialmente la cantidad de estas utilizada para entrenar la red neuronal.

In [11]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/dataset/Train',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='categorical')

testing_dataset = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/dataset/Test',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='categorical')

classifier.fit(training_dataset,
                        steps_per_epoch= len(training_dataset),
                        epochs=50,
                        validation_data=testing_dataset,
                        validation_steps=len(testing_dataset))

Found 900 images belonging to 4 classes.
Found 225 images belonging to 4 classes.
Epoch 1/50
29/29 [==============================] - 280s 10s/step - loss: 0.8960 - accuracy: 0.6167 - val_loss: 0.7718 - val_accuracy: 0.6667
Epoch 2/50
29/29 [==============================] - 11s 376ms/step - loss: 0.6616 - accuracy: 0.7389 - val_loss: 0.5727 - val_accuracy: 0.8578
Epoch 3/50
29/29 [==============================] - 11s 385ms/step - loss: 0.5431 - accuracy: 0.7956 - val_loss: 0.4483 - val_accuracy: 0.8444
Epoch 4/50
29/29 [==============================] - 11s 375ms/step - loss: 0.5729 - accuracy: 0.7711 - val_loss: 0.4645 - val_accuracy: 0.9156
Epoch 5/50
29/29 [==============================] - 11s 377ms/step - loss: 0.4527 - accuracy: 0.8300 - val_loss: 0.3925 - val_accuracy: 0.9156
Epoch 6/50
29/29 [==============================] - 11s 387ms/step - loss: 0.4065 - accuracy: 0.8378 - val_loss: 0.4134 - val_accuracy: 0.8844
Epoch 7/50
29/29 [==============================] - 11s 386ms

Se observa un excelente comportamiento de la red neuronal, la cual ha obtenido un accuracy de 0.9811 en el conjunto de entrenamiento y un 0.9422 en el de validación.